In [1]:
import gym

In [3]:
env = gym.make('CartPole-v0')

for i_episode in range(20):

    observation = env.reset()

    for t in range(100):

        env.render()

        print(observation)

        action = env.action_space.sample()

        observation, reward, done, info = env.step(action)

        if done:

            print("Episode finished after {} timesteps".format(t+1))

            break

[-0.02869297 -0.00879141 -0.03329654 -0.0344095 ]
[-0.0288688  -0.20342045 -0.03398473  0.24758486]
[-0.03293721 -0.39804098 -0.02903304  0.5293575 ]
[-0.04089803 -0.20252287 -0.01844588  0.22766942]
[-0.04494848 -0.39737642 -0.0138925   0.51447725]
[-0.05289601 -0.5923     -0.00360295  0.8027501 ]
[-0.06474201 -0.78737235  0.01245205  1.0942975 ]
[-0.08048946 -0.59241664  0.034338    0.8055474 ]
[-0.09233779 -0.3977818   0.05044895  0.52386063]
[-0.10029343 -0.2034048   0.06092616  0.24749179]
[-0.10436153 -0.39934158  0.065876    0.55875283]
[-0.11234836 -0.5953234   0.07705105  0.8714414 ]
[-0.12425482 -0.79140395  0.09447988  1.1873202 ]
[-0.1400829  -0.5976254   0.11822629  0.9256845 ]
[-0.1520354  -0.4042814   0.13673998  0.6723707 ]
[-0.16012104 -0.60101247  0.15018739  1.0047891 ]
[-0.17214128 -0.79778636  0.17028317  1.3406166 ]
[-0.18809702 -0.60516715  0.1970955   1.1056883 ]
Episode finished after 18 timesteps
[ 0.02697626 -0.04482434 -0.00709755  0.0287788 ]
[ 0.02607977  

[ 0.02351628 -0.01063676 -0.12580068 -0.41378573]
[ 0.02330355  0.1860229  -0.1340764  -0.74333197]
[ 0.02702401 -0.00701883 -0.14894304 -0.49566862]
[ 0.02688363  0.18985495 -0.1588564  -0.8313373 ]
[ 0.03068073 -0.00278085 -0.17548315 -0.5925295 ]
[ 0.03062511 -0.19506842 -0.18733375 -0.35985538]
[ 0.02672374  0.00215369 -0.19453084 -0.7052673 ]
[ 0.02676682 -0.18981718 -0.2086362  -0.47957432]
Episode finished after 26 timesteps
[ 0.04405732 -0.00210715 -0.02725067  0.04670385]
[ 0.04401517 -0.19682796 -0.02631659  0.33066592]
[ 0.04007861 -0.39156562 -0.01970327  0.61493504]
[ 0.0322473  -0.58640677 -0.00740457  0.9013477 ]
[ 0.02051917 -0.7814276   0.01062238  1.191694  ]
[ 0.00489061 -0.9766856   0.03445626  1.4876873 ]
[-0.0146431  -0.78199995  0.06421001  1.2059604 ]
[-0.0302831  -0.97789013  0.08832922  1.518055  ]
[-0.0498409  -1.1739624   0.11869031  1.8369516 ]
[-0.07332014 -1.3701794   0.15542935  2.1640196 ]
[-0.10072374 -1.1768825   0.19870974  1.923082  ]
Episode finish

[ 0.16731535 -0.41140088  0.19384573  1.4901083 ]
Episode finished after 49 timesteps
[ 0.01601327 -0.0110404  -0.04776662  0.04414306]
[ 0.01579246  0.1847328  -0.04688376 -0.26321983]
[ 0.01948712  0.38049152 -0.05214816 -0.57031405]
[ 0.02709695  0.57630455 -0.06355444 -0.8789589 ]
[ 0.03862304  0.7722298  -0.08113361 -1.1909258 ]
[ 0.05406764  0.5782474  -0.10495213 -0.92473656]
[ 0.06563259  0.7746184  -0.12344687 -1.2484716 ]
[ 0.08112495  0.97108763 -0.1484163  -1.577134  ]
[ 0.10054671  0.7780131  -0.17995897 -1.3341805 ]
[ 0.11610697  0.585557   -0.20664258 -1.1027812 ]
Episode finished after 10 timesteps
[ 0.03203548 -0.04512389  0.03239699  0.04305115]
[ 0.031133    0.14951889  0.03325801 -0.23923676]
[ 0.03412338 -0.046062    0.02847328  0.0637484 ]
[ 0.03320214 -0.24158037  0.02974824  0.36527708]
[ 0.02837054 -0.04689354  0.03705379  0.08212073]
[ 0.02743266 -0.24252652  0.0386962   0.3862601 ]
[ 0.02258213 -0.43817583  0.0464214   0.6908882 ]
[ 0.01381862 -0.6339101   0.